# Limpieza generación de características
Este notebook contiene los pasos que se usan para el tratamiento inicial de los datos, con el fin de obtener un dataset numérico para posteriormente, hacer una red neuronal tradicional y predecir el precio de los apartamentos

In [1]:
import pandas as pd
import os

# Carga el dataset y devuelve un dataframe de Pandas
def load_airbnb_dataset(ruta,nombre):
    csv_path = os.path.join(ruta, nombre)
    return pd.read_csv(csv_path, sep=';')

In [2]:
housing = load_airbnb_dataset('datasets','datasetImagenesFinal.csv')

pd.options.display.max_columns = None
housing.head()

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,Notes,Transit,Access,Interaction,House Rules,Thumbnail Url,Medium Url,Picture Url,XL Picture Url,Host ID,Host URL,Host Name,Host Since,Host Location,Host About,Host Response Time,Host Response Rate,Host Acceptance Rate,Host Thumbnail Url,Host Picture Url,Host Neighbourhood,Host Listings Count,Host Total Listings Count,Host Verifications,Street,Neighbourhood,Neighbourhood Cleansed,Neighbourhood Group Cleansed,City,State,Zipcode,Market,Smart Location,Country Code,Country,Latitude,Longitude,Property Type,Room Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Square Feet,Price,Weekly Price,Monthly Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Calendar Updated,Has Availability,Availability 30,Availability 60,Availability 90,Availability 365,Calendar last Scraped,Number of Reviews,First Review,Last Review,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features,imagenLocal
0,11210388,https://www.airbnb.com/rooms/11210388,20170306202425,2017-03-07,The Loft-Full Bath-Deck w/View,Loft in the Hill country 12-15 minutes directl...,This loft has a spectacular view of the hills ...,Loft in the Hill country 12-15 minutes directl...,none,This neighborhood is located in the hills west...,The loft is located on a gated property. Ther...,There is no public transportation from this lo...,The loft is entirely private to the guest as i...,My wife and I live on the property so we would...,The loft is entirely private but is adjacent t...,https://a0.muscache.com/im/pictures/c351a9b5-5...,https://a0.muscache.com/im/pictures/c351a9b5-5...,https://public.opendatasoft.com/api/v2/catalog...,https://a0.muscache.com/im/pictures/c351a9b5-5...,58386783,https://www.airbnb.com/users/show/58386783,John,2016-02-11,My father was USAF so I was born at Offutt AFB...,I am a High Tech Professional with over 30 yea...,within an hour,100.0,NaN,https://a0.muscache.com/im/pictures/c5c78f08-f...,https://a0.muscache.com/im/pictures/c5c78f08-f...,NaN,1.0,1.0,"email,phone,reviews,kba","Canyon Edge Drive, Austin, TX 78733, United St...",NaN,78733,NaN,Austin,TX,78733,Austin,"Austin, TX",US,United States,30.337361,-97.863277,Loft,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"TV,Cable TV,Internet,Wireless Internet,Air con...",NaN,58.0,NaN,NaN,100.0,NaN,1,0,2,1125,today,NaN,10,34,64,281,2017-03-06,42,2016-03-13,2017-02-26,98.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,NaN,moderate,1.0,3.50,"30.3373609355,-97.8632766782","Host Is Superhost,Host Has Profile Pic,Host Id...",img\11210388.jpg
1,17471131,https://www.airbnb.com/rooms/17471131,20170407214050,2017-04-08,"Claris I, Friendly Rentals","This apartment has: 1 double bed, 1 double sof...","This apartment has: 1 double bed, 1 double sof...","This apartment has: 1 double bed, 1 double sof...",none,Plaza Cataluña Catalonia Square is the large...,NaN,Ideal to discover the city either on foot or b...,Travellers will have access to the entire apar...,We will be more than happy to help you with an...,CHECK-IN Week Days: The check-in and key colle...,https://a0.muscache.com/im/pictures/3ce27aee-f...,https://a0.muscache.com/im/pictures/3ce27aee-f...,https://public.opendatasoft.com/api/v2/catalog...,https://a0.muscache.com/im/pictures/3ce27aee-f...,102947901,https://www.airbnb.com/users/show/102947901,Claudia,2016-11-08,"Barcelona, Catalonia, Spain",Hola a todos! Soy Claudia Castillo y soy parte...,within an hour,100.0,NaN,https://a0.muscache.com/im/pictures/c071e13e-5...,https://a0.muscache.com/im/pictures/c071e13e-5...,NaN,162.0,162.0,"email,phone,reviews,work_email","Barcelona, Barcelona 08010, Spain",NaN,la Dreta de l'Eixample,Eixample,Barcelona,Barcelona,0801

In [3]:
housing["Price"].isnull().values.sum()

9

In [4]:
housing = housing[housing['Price'].notnull()]

## División del dataset

In [5]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size = 0.2, random_state = 42)

# Guardo en csv la separación
train_set.to_csv('./datasets/train.csv', sep=';', decimal='.', index=False)
test_set.to_csv('./datasets/test.csv', sep=';', decimal='.', index=False)

In [23]:
train_set = load_airbnb_dataset('datasets', 'train.csv')
copTrain = train_set.copy()

## Limpieza y Nuevas variables

In [24]:
columnasNoImport = ['Scrape ID', 'Host ID', 'Longitude', 'Latitude', 'Host Listings Count', 'Host Total Listings Count', 'Calculated host listings count', 'Weekly Price', 'Monthly Price','Availability 30', 'Availability 60', 'Availability 90','Listing Url', 'Last Scraped', 'Name', 'Summary', 'Space', 'Description', 'Neighborhood Overview', 'Notes', 'Transit', 'Access', 'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url', 'Host URL', 'Host Name', 'Host About', 'Host Thumbnail Url', 'Host Picture Url', 'Calendar last Scraped', 'Geolocation']

copTrain.drop(columnasNoImport, axis=1, inplace=True)

In [25]:
copTrain["beds_per_bedrooms"] = copTrain["Beds"] / copTrain["Bedrooms"]
copTrain["Bathrooms_per_bed"] = copTrain["Bathrooms"] / copTrain["Beds"]

In [26]:
# Vamos a ver antes los valores únicos que hay para saber qué columnas tenemos que crear y depaso eliminamos los nan
listaFeatures = []

for x in  copTrain["Features"].dropna().unique():
    for y in str(x).split(","):    
        if y not in listaFeatures:
            listaFeatures.append(y)

def getNewColumnFromFeature(pTexToSearch, pList):
    if pTexToSearch in str(pList).split(","):        
        return 1
    else:
        return 0

for i in listaFeatures:
    copTrain[i] = copTrain["Features"].apply(lambda x: getNewColumnFromFeature(i, x))    

copTrain.drop(['Features'], axis=1, inplace=True)

listaAmenities = []

for x in  copTrain["Amenities"].dropna().unique():
    for y in str(x).split(","):    
        if y not in listaAmenities:
            listaAmenities.append(y)

listaHost = []

for x in  copTrain["Host Verifications"].dropna().unique():
    for y in str(x).split(","):    
        if y not in listaHost:
            listaHost.append(y)

for i in listaAmenities:
    copTrain[i] = copTrain["Amenities"].apply(lambda x: getNewColumnFromFeature(i, x))    

copTrain.drop(['Amenities'], axis=1, inplace=True)

for i in listaHost:
    copTrain[i] = copTrain["Host Verifications"].apply(lambda x: getNewColumnFromFeature(i, x))    

copTrain.drop(['Host Verifications'], axis=1, inplace=True)

In [27]:
columnasNoImport = ['Other pet(s)','Room-darkening shades','Lockbox','Window guards','Babysitter recommendations','Crib','Elevator in Building','Smoke Detector','Carbon Monoxide Detector','Free Parking on Premises','Safety Card','Fire Extinguisher','Hot Tub','Laptop Friendly Workspace','Pack ’n Play/travel crib','Smoking Allowed','Indoor Fireplace','Family/Kid Friendly','First Aid Kit','High chair','Keypad','Game console','Smartlock','Outlet covers','Children’s books and toys','Baby bath','Table corner guards','Air Conditioning','24-Hour Check-in','Hair Dryer','Buzzer/Wireless Intercom','Children’s dinnerware','Pets Allowed','Paid parking off premises','Stair gates','Wheelchair Accessible','Washer / Dryer','Hot water','Bed linens','Extra pillows and blankets','Microwave','Coffee maker','Refrigerator','Dishwasher','Dishes and silverware','Cooking basics','Oven','Patio or balcony','Garden or backyard','Luggage dropoff allowed','Long term stays allowed','Path to entrance lit at night','Changing table','Free parking on street','Lock on Bedroom Door','EV charger','BBQ grill','Suitable for Events','Fireplace guards','manual_offline','selfie','identity_manual','sent_id','photographer','None','weibo','amex','Street', 'Neighbourhood', 'Neighbourhood Cleansed', 'Neighbourhood Group Cleansed', 'State', 'Market', 'Smart Location', 'Country Code', 'Country','Calendar Updated', 'First Review', 'Last Review', 'License', 'Host Neighbourhood']

for col in columnasNoImport:
    if col in copTrain.columns:
        copTrain.drop(col, axis=1, inplace=True)

In [28]:
# Borramos todas las filas que no correspondan a Madrid y las que tengan códigos postales erróneos
copTrain = copTrain[copTrain['City'] == 'Madrid']
copTrain = copTrain[copTrain['Zipcode'].apply(lambda x: len(str(x)) == 5)]
copTrain.drop(["City"], axis=1, inplace=True)

# Experiences Offered no tiene valor, la borramos
copTrain.drop(["Experiences Offered"], axis=1, inplace=True)

# Borramos los missings de Host Since
copTrain = copTrain[copTrain['Host Since'].isnull() == False]

from datetime import datetime

# Para ello, nos creamos una nueva columna que contenga el dato convertido a fecha
copTrain['Host Since'] = copTrain['Host Since'].apply(lambda x: datetime.strptime(str(x),'%Y-%m-%d'))

# Generamos nueva columna y borramos la original
copTrain['Host Years No.'] = copTrain['Host Since'].apply(lambda x: 2017 - x.year)
copTrain.drop(['Host Since'], axis=1, inplace=True)

copTrain.drop(['Host Location'], axis=1, inplace=True)

# Son todo missings, eliminamos la columna
copTrain.drop(['Host Acceptance Rate'], axis=1, inplace=True)

# Pasa lo mismo con la columna Has Availability
copTrain.drop(['Has Availability'], axis=1, inplace=True)

# Pasa lo mismo con la columna Jurisdiction Names
copTrain.drop(['Jurisdiction Names'], axis=1, inplace=True)

## Tratamiento de NaN

In [29]:
# Rellenamos los missings con el valor más frecuente
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="most_frequent")
imputer.fit(copTrain) # Entrenamos el imputador
X = imputer.transform(copTrain) # Transormamos según el imputador entrenado

copTrain = pd.DataFrame(X, columns=copTrain.columns, index=copTrain.index) # Reconvertimos a de numpy a pandas

In [30]:
# Convertir pies cuadrados a metros cuadrados
copTrain['M2'] = (copTrain['Square Feet'] * 0.092903)
copTrain.drop(['Square Feet'], axis=1, inplace=True)

## Variables categóricas
Convertir las variables categóricas a números. Las variables que se transformarán son las siguientes:

- Host Response Time
- Property Type
- Room Type
- Bed Type
- Cancellation Policy

In [31]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
copTrain['Host Response Time'] = ordinal_encoder.fit_transform(copTrain['Host Response Time'].values.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
copTrain['Property Type'] = ordinal_encoder.fit_transform(copTrain['Property Type'].values.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
copTrain['Room Type'] = ordinal_encoder.fit_transform(copTrain['Room Type'].values.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
copTrain['Bed Type'] = ordinal_encoder.fit_transform(copTrain['Bed Type'].values.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
copTrain['Cancellation Policy'] = ordinal_encoder.fit_transform(copTrain['Cancellation Policy'].values.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
copTrain['Zipcode'] = ordinal_encoder.fit_transform(copTrain['Zipcode'].values.reshape(-1,1))

In [32]:
import numpy as np
from numpy import inf

copTrain = copTrain.replace([np.inf, -np.inf], 0)

## Generar nuevo dataset limpio
Una vez hecha la limpieza, guardo el resultado en un nuevo csv y genero otro con la columna de la imagen que se usará más adelante

In [34]:
nuevoTrainImagenes = copTrain[['ID', 'imagenLocal']].copy()
nuevoTrainImagenes.to_csv('./datasets/imagenesTrainFinal.csv', sep=';', decimal='.', index=False)

columnasNoImport = ['ID', 'imagenLocal']
copTrain = copTrain.drop(columnasNoImport, axis=1, inplace=False)
copTrain.to_csv('./datasets/trainFinal.csv', sep=';', decimal='.', index=False)

## Aplicar transformaciones a datos de test

In [37]:
test_set = load_airbnb_dataset('datasets', 'test.csv')
copTest = test_set.copy()

columnasNoImport = ['Scrape ID', 'Host ID', 'Longitude', 'Latitude', 'Host Listings Count', 'Host Total Listings Count', 'Calculated host listings count', 'Weekly Price', 'Monthly Price','Availability 30', 'Availability 60', 'Availability 90','Listing Url', 'Last Scraped', 'Name', 'Summary', 'Space', 'Description', 'Neighborhood Overview', 'Notes', 'Transit', 'Access', 'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url', 'Host URL', 'Host Name', 'Host About', 'Host Thumbnail Url', 'Host Picture Url', 'Calendar last Scraped', 'Geolocation']

copTest.drop(columnasNoImport, axis=1, inplace=True)

copTest["beds_per_bedrooms"] = copTest["Beds"] / copTest["Bedrooms"]
copTest["Bathrooms_per_bed"] = copTest["Bathrooms"] / copTest["Beds"]

listaFeatures = []

for x in  copTest["Features"].dropna().unique():
    for y in str(x).split(","):    
        if y not in listaFeatures:
            listaFeatures.append(y)

def getNewColumnFromFeature(pTexToSearch, pList):
    if pTexToSearch in str(pList).split(","):        
        return 1
    else:
        return 0

for i in listaFeatures:
    copTest[i] = copTest["Features"].apply(lambda x: getNewColumnFromFeature(i, x))    

copTest.drop(['Features'], axis=1, inplace=True)

listaAmenities = []

for x in  copTest["Amenities"].dropna().unique():
    for y in str(x).split(","):    
        if y not in listaAmenities:
            listaAmenities.append(y)

listaHost = []

for x in  copTest["Host Verifications"].dropna().unique():
    for y in str(x).split(","):    
        if y not in listaHost:
            listaHost.append(y)

for i in listaAmenities:
    copTest[i] = copTest["Amenities"].apply(lambda x: getNewColumnFromFeature(i, x))    

copTest.drop(['Amenities'], axis=1, inplace=True)

for i in listaHost:
    copTest[i] = copTest["Host Verifications"].apply(lambda x: getNewColumnFromFeature(i, x))    

copTest.drop(['Host Verifications'], axis=1, inplace=True)

columnasNoImport = ['Other pet(s)','Room-darkening shades','Lockbox','Window guards','Babysitter recommendations','Crib','Elevator in Building','Smoke Detector','Carbon Monoxide Detector','Free Parking on Premises','Safety Card','Fire Extinguisher','Hot Tub','Laptop Friendly Workspace','Pack ’n Play/travel crib','Smoking Allowed','Indoor Fireplace','Family/Kid Friendly','First Aid Kit','High chair','Keypad','Game console','Smartlock','Outlet covers','Children’s books and toys','Baby bath','Table corner guards','Air Conditioning','24-Hour Check-in','Hair Dryer','Buzzer/Wireless Intercom','Children’s dinnerware','Pets Allowed','Paid parking off premises','Stair gates','Wheelchair Accessible','Washer / Dryer','Hot water','Bed linens','Extra pillows and blankets','Microwave','Coffee maker','Refrigerator','Dishwasher','Dishes and silverware','Cooking basics','Oven','Patio or balcony','Garden or backyard','Luggage dropoff allowed','Long term stays allowed','Path to entrance lit at night','Changing table','Free parking on street','Lock on Bedroom Door','EV charger','BBQ grill','Suitable for Events','Fireplace guards','manual_offline','selfie','identity_manual','sent_id','photographer','None','weibo','amex','Street', 'Neighbourhood', 'Neighbourhood Cleansed', 'Neighbourhood Group Cleansed', 'State', 'Market', 'Smart Location', 'Country Code', 'Country','Calendar Updated', 'First Review', 'Last Review', 'License', 'Host Neighbourhood']

for col in columnasNoImport:
    if col in copTest.columns:
        copTest.drop(col, axis=1, inplace=True)
		
# Borramos todas las filas que no correspondan a Madrid y las que tengan códigos postales erróneos
copTest = copTest[copTest['City'] == 'Madrid']
copTest = copTest[copTest['Zipcode'].apply(lambda x: len(str(x)) == 5)]
copTest.drop(["City"], axis=1, inplace=True)

# Experiences Offered no tiene valor, la borramos
copTest.drop(["Experiences Offered"], axis=1, inplace=True)

# Borramos los missings de Host Since
copTest = copTest[copTest['Host Since'].isnull() == False]

from datetime import datetime

# Para ello, nos creamos una nueva columna que contenga el dato convertido a fecha
copTest['Host Since'] = copTest['Host Since'].apply(lambda x: datetime.strptime(str(x),'%Y-%m-%d'))

# Generamos nueva columna y borramos la original
copTest['Host Years No.'] = copTest['Host Since'].apply(lambda x: 2017 - x.year)
copTest.drop(['Host Since'], axis=1, inplace=True)

copTest.drop(['Host Location'], axis=1, inplace=True)

# Son todo missings, eliminamos la columna
copTest.drop(['Host Acceptance Rate'], axis=1, inplace=True)

# Pasa lo mismo con la columna Has Availability
copTest.drop(['Has Availability'], axis=1, inplace=True)

# Pasa lo mismo con la columna Jurisdiction Names
copTest.drop(['Jurisdiction Names'], axis=1, inplace=True)

# Rellenamos los missings con el valor más frecuente
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="most_frequent")
imputer.fit(copTest) # Entrenamos el imputador
X = imputer.transform(copTest) # Transormamos según el imputador entrenado

copTest = pd.DataFrame(X, columns=copTest.columns, index=copTest.index) # Reconvertimos a de numpy a pandas

# Convertir pies cuadrados a metros cuadrados
copTest['M2'] = (copTest['Square Feet'] * 0.092903)
copTest.drop(['Square Feet'], axis=1, inplace=True)

from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
copTest['Host Response Time'] = ordinal_encoder.fit_transform(copTest['Host Response Time'].values.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
copTest['Property Type'] = ordinal_encoder.fit_transform(copTest['Property Type'].values.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
copTest['Room Type'] = ordinal_encoder.fit_transform(copTest['Room Type'].values.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
copTest['Bed Type'] = ordinal_encoder.fit_transform(copTest['Bed Type'].values.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
copTest['Cancellation Policy'] = ordinal_encoder.fit_transform(copTest['Cancellation Policy'].values.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
copTest['Zipcode'] = ordinal_encoder.fit_transform(copTest['Zipcode'].values.reshape(-1,1))

import numpy as np
from numpy import inf


copTest = copTest.replace([np.inf, -np.inf], 0)

nuevoTestImagenes = copTest[['ID', 'imagenLocal']].copy()
nuevoTestImagenes.to_csv('./datasets/imagenesTestFinal.csv', sep=';', decimal='.', index=False)

columnasNoImport = ['ID', 'imagenLocal']
copTest = copTest.drop(columnasNoImport, axis=1, inplace=False)
copTest.to_csv('./datasets/testFinal.csv', sep=';', decimal='.', index=False)